In [ ]:
#Problem, alles hier in sheet.MMseqs2 WSL form rein

import pandas as pdsudo apt install python3 python3-pip -y




# TSV-Datei einlesen
ab_ag_MMseqs2 = pd.read_csv('data/ab_ag_annotated.tsv', sep='\t')

# NUR die Spalte 'CDR_H1' extrahieren
CDR_H1_seq = ab_ag_MMseqs2['CDR_H1']

with open("CDR_H1_seqs.fasta", "w") as f:
    for i, seq in enumerate(CDR_H1_seq):
        # Wenn seq eine Liste ist, mach sie zu einem String
        if isinstance(seq, list):
            seq = ''.join(seq)
        elif isinstance(seq, str) and seq.startswith("[") and seq.endswith("]"):
            # Falls als String gespeicherte Liste: eval-sicher entschachteln
            seq = ''.join(eval(seq))
        f.write(f">seq{i}\n{seq}\n")

#Kontrolle kann auch wieder entfernt werden
print("Erste 3 FASTA-Sequenzen:")
with open("CDR_H1_seqs.fasta") as f:
    for _ in range(6):  # 3 Sequenzen = 6 Zeilen (je Header + Sequenz)
        print(f.readline().strip())

Erste 3 FASTA-Sequenzen:
>seq0
FTFSAYG
>seq1
FTFSAYG
>seq2
FTFSAYG


In [ ]:
#Problem befehl muss vermutlich in Terminal geschrieben werden

import subprocess
import os

#Datenbank erzeugen
subprocess.run("mmseqs createdb CDR_H1_seqs.fasta cdr_h1_db", shell = True)
#cdr_h1_db ist Name der MMseqs2 internen DAtenbank

#Clustern
subprocess.run("mmseqs easy-cluster cdr_h1_db cdr_h1_clu tmp --min-seq-id 0.9 -c 0.8", shell = True)
#cdr_1_db: Ergebnis-Datenbank mit den Clustern
#tmp: temporärer Arbeitsornder
# --minseq-id 0.9: Mindestens 90% sequenz identität
# -c 0.8: Coverage: 80% der kürzeren Sequenz vom alignment müssen abgedeckt sein

#TSV exportieren
subprocess.run("mmseqs createtsv cdr_h1_db cdr_h1_db cdr_hi_clu cdr_h1_cluster.tsv", shell = True)
#cdr_h1_db: datenbank mit sequenzen (zweimal als Query und als target)
#cdr_h1_clu: Clusterergebnisse
#cdr_h1_cluster.tsv: Name der Ausgabedatei im Textformat

print("Cluster-DB vorhanden:", os.path.exists("cdr_h1_clu"))
print("TSV vorhanden:", os.path.exists("cdr_h1_clusters.tsv"))


result = subprocess.run(
    "mmseqs easy-cluster cdr_h1_db cdr_h1_clu tmp --min-seq-id 0.9 -c 0.8",
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("📤 STDOUT:")
print(result.stdout)

print("📥 STDERR (Fehlerausgabe):")
print(result.stderr)




Cluster-DB vorhanden: False
TSV vorhanden: False
📤 STDOUT:

📥 STDERR (Fehlerausgabe):
Der Befehl "mmseqs" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.



In [ ]:
#Code zum Clustern visualisieren aber noch nicht fertig
#Problem hat cdr_ha1_cluster.tsv noch nicht gefunden, einfacher fix
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

print("Aktuelles Arbeitsverzeichnis:", os.getcwd())
# Lade die Clusterdatei
cluster_CDR_H1 = pd.read_csv("cdr_h1_clusters.tsv", sep="\t", header=None, names=["Cluster", "Member"])

# Liste aller Member-Sequenzen
members = cluster_CDR_H1["Member"].unique()

# Weise zufällige 3D-Koordinaten zu
np.random.seed(42)  # reproduzierbar
coords = np.random.rand(len(members), 3)

# Erzeuge Mapping: Sequenz-ID → Cluster
cluster_map = cluster_CDR_H1.set_index("Member")["Cluster"].to_dict()

# Farbe pro Cluster
clusters = list(set(cluster_map.values()))
cluster_colors = {c: i for i, c in enumerate(clusters)}

# Plot vorbereiten
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

for i, seq_id in enumerate(members):
    x, y, z = coords[i]
    cluster = cluster_map[seq_id]
    color = plt.cm.tab10(cluster_colors[cluster] % 10)
    ax.scatter(x, y, z, color=color, label=cluster if i == 0 else "", s=50)
    ax.text(x, y, z, seq_id, size=8)

ax.set_title("CDR_H1 Cluster Visualisierung (3D)")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.tight_layout()
plt.show()


Aktuelles Arbeitsverzeichnis: c:\Users\chris\Documents\GitHub\group04-team04


FileNotFoundError: [Errno 2] No such file or directory: 'cdr_h1_clusters.tsv'